In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt

# Load MobileNetV2 with pretrained ImageNet weights and exclude the top layers
def create_transfer_model(input_shape, num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze some of the layers
    base_model.trainable = False

    # Add custom layers on top of the base model
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))  # L2 regularization
    model.add(layers.Dropout(0.5))  # Add dropout to reduce overfitting
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # Increased augmentation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data will be used for validation
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/mod_dataset_heartbeat',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/mod_dataset_heartbeat',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Initialize the model
input_shape = (224, 224, 3)
num_classes = len(train_generator.class_indices)
model = create_transfer_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Set up callbacks
checkpoint = ModelCheckpoint('best_transfer_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=6, verbose=1, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=25,  # Increase epochs for better training
    validation_data=validation_generator,
    callbacks=[checkpoint, reduce_lr, early_stopping]
)

# Fine-tune the model by unfreezing more layers
base_model.trainable = True
for layer in base_model.layers[:-50]:  # Unfreeze last 50 layers for fine-tuning
    layer.trainable = False

# Recompile the model with a very low learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),  # Fine-tune with a smaller learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    epochs=10,  # Few additional epochs for fine-tuning
    validation_data=validation_generator,
    callbacks=[checkpoint, reduce_lr, early_stopping]
)

# Plot training and validation accuracy and loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(fine_tune_history.history['accuracy'], label='Fine-tuned Training Accuracy')
plt.plot(fine_tune_history.history['val_accuracy'], label='Fine-tuned Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(fine_tune_history.history['loss'], label='Fine-tuned Training Loss')
plt.plot(fine_tune_history.history['val_loss'], label='Fine-tuned Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

print('Finished Training')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/mod_dataset_heartbeat'